In [2]:
from google.colab import files
import pandas as pd
import shutil
import os

uploaded = files.upload()

file_name = next(iter(uploaded))
target_path = '/content/bc_trip259172515_230215.csv'
shutil.move(file_name, target_path)





Saving bc_trip259172515_230215.csv to bc_trip259172515_230215.csv


'/content/bc_trip259172515_230215.csv'

In [3]:
df = pd.read_csv(target_path)
print("Number of breadcrumb records:", len(df))
print(df.head())

Number of breadcrumb records: 161
   EVENT_NO_TRIP  EVENT_NO_STOP            OPD_DATE  VEHICLE_ID  METERS  \
0      259172515      259172517  15FEB2023:00:00:00        4223      40   
1      259172515      259172517  15FEB2023:00:00:00        4223      48   
2      259172515      259172517  15FEB2023:00:00:00        4223      57   
3      259172515      259172517  15FEB2023:00:00:00        4223      73   
4      259172515      259172517  15FEB2023:00:00:00        4223     112   

   ACT_TIME  GPS_LONGITUDE  GPS_LATITUDE  GPS_SATELLITES  GPS_HDOP  
0     20469    -122.648137     45.493082              12       0.7  
1     20474    -122.648240     45.493070              12       0.8  
2     20479    -122.648352     45.493123              12       0.8  
3     20484    -122.648385     45.493262              12       0.7  
4     20489    -122.648347     45.493582              12       0.8  


In [4]:
drop_tables = df.drop(columns=['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'])
print(drop_tables.head())

   EVENT_NO_TRIP            OPD_DATE  VEHICLE_ID  METERS  ACT_TIME  \
0      259172515  15FEB2023:00:00:00        4223      40     20469   
1      259172515  15FEB2023:00:00:00        4223      48     20474   
2      259172515  15FEB2023:00:00:00        4223      57     20479   
3      259172515  15FEB2023:00:00:00        4223      73     20484   
4      259172515  15FEB2023:00:00:00        4223     112     20489   

   GPS_LONGITUDE  GPS_LATITUDE  
0    -122.648137     45.493082  
1    -122.648240     45.493070  
2    -122.648352     45.493123  
3    -122.648385     45.493262  
4    -122.648347     45.493582  


In [5]:
filter = pd.read_csv(
    '/content/bc_trip259172515_230215.csv',
    usecols=lambda col: col not in ['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP']
)

print(filter.head())

   EVENT_NO_TRIP            OPD_DATE  VEHICLE_ID  METERS  ACT_TIME  \
0      259172515  15FEB2023:00:00:00        4223      40     20469   
1      259172515  15FEB2023:00:00:00        4223      48     20474   
2      259172515  15FEB2023:00:00:00        4223      57     20479   
3      259172515  15FEB2023:00:00:00        4223      73     20484   
4      259172515  15FEB2023:00:00:00        4223     112     20489   

   GPS_LONGITUDE  GPS_LATITUDE  
0    -122.648137     45.493082  
1    -122.648240     45.493070  
2    -122.648352     45.493123  
3    -122.648385     45.493262  
4    -122.648347     45.493582  


In [11]:

filter['OPD_DATE'] = pd.to_datetime(filter['OPD_DATE'], format='%d%b%Y:%H:%M:%S')

filter['ACT_TIME'] = pd.to_timedelta(filter['ACT_TIME'], unit='s')

filter['TIMESTAMP'] = filter['OPD_DATE'] + filter['ACT_TIME']

print(filter[['OPD_DATE', 'ACT_TIME', 'TIMESTAMP']].head())


print(filter.head())

    OPD_DATE        ACT_TIME           TIMESTAMP
0 2023-02-15 0 days 05:41:09 2023-02-15 05:41:09
1 2023-02-15 0 days 05:41:14 2023-02-15 05:41:14
2 2023-02-15 0 days 05:41:19 2023-02-15 05:41:19
3 2023-02-15 0 days 05:41:24 2023-02-15 05:41:24
4 2023-02-15 0 days 05:41:29 2023-02-15 05:41:29
   EVENT_NO_TRIP   OPD_DATE  VEHICLE_ID  METERS        ACT_TIME  \
0      259172515 2023-02-15        4223      40 0 days 05:41:09   
1      259172515 2023-02-15        4223      48 0 days 05:41:14   
2      259172515 2023-02-15        4223      57 0 days 05:41:19   
3      259172515 2023-02-15        4223      73 0 days 05:41:24   
4      259172515 2023-02-15        4223     112 0 days 05:41:29   

   GPS_LONGITUDE  GPS_LATITUDE           TIMESTAMP  
0    -122.648137     45.493082 2023-02-15 05:41:09  
1    -122.648240     45.493070 2023-02-15 05:41:14  
2    -122.648352     45.493123 2023-02-15 05:41:19  
3    -122.648385     45.493262 2023-02-15 05:41:24  
4    -122.648347     45.493582 2023-02

In [13]:
filter = filter.drop(columns=['OPD_DATE', 'ACT_TIME'])

print(filter.head())

   EVENT_NO_TRIP  VEHICLE_ID  METERS  GPS_LONGITUDE  GPS_LATITUDE  \
0      259172515        4223      40    -122.648137     45.493082   
1      259172515        4223      48    -122.648240     45.493070   
2      259172515        4223      57    -122.648352     45.493123   
3      259172515        4223      73    -122.648385     45.493262   
4      259172515        4223     112    -122.648347     45.493582   

            TIMESTAMP  
0 2023-02-15 05:41:09  
1 2023-02-15 05:41:14  
2 2023-02-15 05:41:19  
3 2023-02-15 05:41:24  
4 2023-02-15 05:41:29  


In [17]:
filter['dM'] = filter['METERS'].diff()
filter['dT'] = filter['TIMESTAMP'].diff().dt.total_seconds()

filter['SPEED'] = filter.apply(
    lambda row: row['dM'] / row['dT'] if row['dT'] and row['dT'] != 0 else 0,
    axis=1
)

filter.drop(columns=['dM', 'dT'], inplace=True)


print(f"Minimum Speed:",filter['SPEED'].min())
print(f"Maximum Speed:",filter['SPEED'].max())
print(f"Average Speed:",  filter['SPEED'].mean())

Minimum Speed: 0.0
Maximum Speed: 17.4
Average Speed: 7.227205815018314


In [36]:
from google.colab import files
import pandas as pd

uploaded = files.upload()
large_df = pd.read_csv("bc_veh4223_230215.csv")


Saving bc_veh4223_230215.csv to bc_veh4223_230215 (7).csv


In [40]:
large_df['OPD_DATE'] = pd.to_datetime(large_df['OPD_DATE'], format='%d%b%Y:%H:%M:%S')

large_df['TIMESTAMP'] = large_df['OPD_DATE'] + pd.to_timedelta(large_df['ACT_TIME'], unit='s')

large_df['dM'] = large_df['METERS'].diff()
large_df['dT'] = large_df['TIMESTAMP'].diff()

large_df['SPEED'] = large_df.apply(
    lambda row: row['dM'] / row['dT'].total_seconds() if pd.notna(row['dT']) else None,
    axis=1
)

trip = large_df.groupby('EVENT_NO_TRIP')['SPEED'].agg(['max', 'median'])

max_speed = trip.loc[trip['max'].idxmax()]
speed_value = max_speed['max']

max_record = large_df[large_df['SPEED'] == speed_value].iloc[0]

print(f"Max speed for vehicle #4223 on Feb 15, 2023 was", max_speed)
print(f"The Location ,Longitude, Latitude and time of max speed:", max_record['GPS_LONGITUDE'], max_record['GPS_LATITUDE'], max_record['TIMESTAMP'])
print(f"Median speed for vehicle #4223 on February 15 2023: {trip['median'].median()} m/s")



Max speed for vehicle #4223 on Feb 15, 2023 was max       17.4
median     6.4
Name: 259172515, dtype: float64
The Location ,Longitude, Latitude and time of max speed: -122.660822 45.505452 2023-02-15 05:44:49
Median speed for vehicle #4223 on February 15 2023: 7.2 m/s
